In [1]:
original_exp_44 = 'livestalk-17'
original_exp_44_45 = 'livestalk-17'
original_exp_44_48 = 'livestalk-17'
original_exp_carbondale = 'livestalk_2-7'
holdout_basalt = 'livestalk_3-1' 

new_exp_44 = 'livestalk-connolly-raw-44-h'
new_exp_44_45 = 'livestalk-connolly-raw-44_45-h'
new_exp_44_48 = 'livestalk-connolly-raw-44_48-h'
new_exp_carbondale = 'livestalk_carbondale-raw-h'

In [2]:
from shutil import copyfile
import os
import glob
import pandas as pd

In [3]:
def make_folder(path):
    if not os.path.exists(path):
        os.mkdir(path)
        os.mkdir(os.path.join(path,'train'))
        os.mkdir(os.path.join(path,'train','images'))
        os.mkdir(os.path.join(path,'train','labels'))

        os.mkdir(os.path.join(path,'valid'))
        os.mkdir(os.path.join(path,'valid','images'))
        os.mkdir(os.path.join(path,'valid','labels'))
        
        os.mkdir(os.path.join(path,'test'))
        os.mkdir(os.path.join(path,'test','images'))
        os.mkdir(os.path.join(path,'test','labels'))        
        
        print(f'folders created: {path}')
        
    else:
        print(f'a folder already exists here: {path}')

### Identify folder for original and resplit

In [4]:
original_exp_44_path = os.path.join(os.getcwd(),original_exp_44)
original_exp_44_45_path = os.path.join(os.getcwd(),original_exp_44_45)
original_exp_44_48_path = os.path.join(os.getcwd(),original_exp_44_48)
original_exp_carbondale_path = os.path.join(os.getcwd(),original_exp_carbondale)
holdout_basalt_path = os.path.join(os.getcwd(),holdout_basalt)


new_exp_44_path = os.path.join(os.getcwd(),new_exp_44)
new_exp_44_45_path = os.path.join(os.getcwd(),new_exp_44_45)
new_exp_44_48_path = os.path.join(os.getcwd(),new_exp_44_48)
new_exp_carbondale_path = os.path.join(os.getcwd(),new_exp_carbondale)

make_folder(new_exp_44_path)
make_folder(new_exp_44_45_path)
make_folder(new_exp_44_48_path)
make_folder(new_exp_carbondale_path)


yaml_txt = """names:
- cow
nc: 1
train: {0}/train/images
val: {0}/valid/images
test: {0}/test/images
"""

new_paths = [new_exp_44_path,new_exp_44_45_path,new_exp_44_48_path,new_exp_carbondale_path]
for new_path in new_paths:
    
    with open(os.path.join(new_path,'data.yaml'),'w') as f:
        f.write(yaml_txt.format(new_path))


folders created: /home/omar/datasci/w251-DL/livestalk/livestalk-connolly-raw-44-h
folders created: /home/omar/datasci/w251-DL/livestalk/livestalk-connolly-raw-44_45-h
folders created: /home/omar/datasci/w251-DL/livestalk/livestalk-connolly-raw-44_48-h
folders created: /home/omar/datasci/w251-DL/livestalk/livestalk_carbondale-raw-h


### Get counts in old folders

In [5]:
def get_counts(path,ret=True):
    
    train_images = glob.glob(os.path.join(path,'train','images','*.jpg'))
    train_labels = glob.glob(os.path.join(path,'train','labels','*.txt'))

    valid_images = glob.glob(os.path.join(path,'valid','images','*.jpg'))
    valid_labels = glob.glob(os.path.join(path,'valid','labels','*.txt'))

    test_images = glob.glob(os.path.join(path,'test','images','*.jpg'))
    test_labels = glob.glob(os.path.join(path,'test','labels','*.txt'))


    all_images = train_images + valid_images + test_images
    all_labels = train_labels + valid_labels + test_labels

    all_images = sorted(all_images)
    all_labels = sorted(all_labels)

    assert len(all_images) == len(all_labels)

    print(f'Training: {len(train_images)}')
    print(f'Validation: {len(valid_images)}')
    print(f'Test: {len(test_images)}')
    print(f'Total number of images: {len(all_images)}')
    
    return all_images,all_labels
    
original_exp_44_images, original_exp_44_labels = get_counts(original_exp_44_path)
original_exp_44_images = [x for x in original_exp_44_images if any(y in x.split('_',2)[1] for y in ['0044'])]
original_exp_44_labels = [x for x in original_exp_44_labels if any(y in x.split('_',2)[1] for y in ['0044'])]
print(f'New Number of images in 44: {len(original_exp_44_images)}')

original_exp_44_45_images, original_exp_44_45_labels = get_counts(original_exp_44_45_path)
original_exp_44_45_images = [x for x in original_exp_44_45_images if any(y in x.split('_',2)[1] for y in ['0044','0045'])]
original_exp_44_45_labels = [x for x in original_exp_44_45_labels if any(y in x.split('_',2)[1] for y in ['0044','0045'])]
print(f'New Number of images in 44-45: {len(original_exp_44_45_images)}')

original_exp_44_48_images, original_exp_44_48_labels = get_counts(original_exp_44_48_path)
original_exp_carbondale_images, original_exp_carbondale_labels = get_counts(original_exp_carbondale_path)
holdout_basalt_images, holdout_basalt_labels = get_counts(holdout_basalt_path)

Training: 739
Validation: 208
Test: 190
Total number of images: 1137
New Number of images in 44: 439
Training: 739
Validation: 208
Test: 190
Total number of images: 1137
New Number of images in 44-45: 701
Training: 739
Validation: 208
Test: 190
Total number of images: 1137
Training: 183
Validation: 75
Test: 0
Total number of images: 258
Training: 47
Validation: 12
Test: 0
Total number of images: 59


### Copy files to new location

In [6]:

def copy_files(images,labels,new_path,location):
    assert len(images) == len(labels)
    
    images = sorted(images)
    labels = sorted(labels)
    
    for image,label in zip(images,labels):
        
        image_file = image.rsplit('/',1)[-1].rsplit('.',1)[0]
        label_file = label.rsplit('/',1)[-1].rsplit('.',1)[0]
    
        assert image_file == label_file, 'lists not aligned'
    
        new_image_path = os.path.join(new_path,location,'images',image.rsplit('/',1)[-1])
        new_label_path = os.path.join(new_path,location,'labels',label.rsplit('/',1)[-1])
        
        copyfile(image,new_image_path)
        copyfile(label,new_label_path)
    
        #if valid, copy to test as well
        if location == 'valid':
            test_image_path = os.path.join(new_path,'test','images',image.rsplit('/',1)[-1])
            test_label_path = os.path.join(new_path,'test','labels',label.rsplit('/',1)[-1])        
            copyfile(image,test_image_path)
            copyfile(label,test_label_path)
                                      

#copy training images
copy_files(original_exp_44_images, original_exp_44_labels,new_exp_44_path,location='train')
copy_files(original_exp_44_45_images, original_exp_44_45_labels,new_exp_44_45_path,location='train')
copy_files(original_exp_44_48_images, original_exp_44_48_labels,new_exp_44_48_path,location='train')
copy_files(original_exp_carbondale_images, original_exp_carbondale_labels,new_exp_carbondale_path,location='train')

#copy test images
copy_files(holdout_basalt_images, holdout_basalt_labels,new_exp_44_path,location='valid')
copy_files(holdout_basalt_images, holdout_basalt_labels,new_exp_44_45_path,location='valid')
copy_files(holdout_basalt_images, holdout_basalt_labels,new_exp_44_48_path,location='valid')
copy_files(holdout_basalt_images, holdout_basalt_labels,new_exp_carbondale_path,location='valid')


### Get counts in new location

In [7]:
print('Counts for new_exp_44:')
_, _ = get_counts(new_exp_44_path)
print('\nCounts for new_exp_44_45:')
_, _ = get_counts(new_exp_44_45_path)
print('\nCounts for new_exp_44_48:')
_, _ = get_counts(new_exp_44_48_path)
print('\nCounts for new_exp_carbondale:')
_, _ = get_counts(new_exp_carbondale_path)

Counts for new_exp_44:
Training: 439
Validation: 59
Test: 59
Total number of images: 557

Counts for new_exp_44_45:
Training: 701
Validation: 59
Test: 59
Total number of images: 819

Counts for new_exp_44_48:
Training: 1137
Validation: 59
Test: 59
Total number of images: 1255

Counts for new_exp_carbondale:
Training: 258
Validation: 59
Test: 59
Total number of images: 376
